# SVC-test av augmentering


## Importera nödvändiga paket.

In [1]:
# Av eget intresse vill jag gärna veta hur lång tid olika saker tar.
import time
notebook_start = time.time()  
t0 = time.time()

# Skippa varningar
import warnings

# Boosta prestandan
from sklearnex import patch_sklearn
patch_sklearn()

# Paket för datahantering
import numpy as np
import pandas as pd
import os
import math

# Dataset och modeller
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split #, GridSearchCV, cross_val_score

# Preprocessing/pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.kernel_approximation import Nystroem

# Dimensionsreducering
from sklearn.decomposition import PCA

# Feature-bearbetning
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog

# Data augmentation och förbehandling - stabil version för SciPy
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter, map_coordinates

# Bildbehandling
from PIL import Image

# Modeller 
from sklearn.svm import SVC, LinearSVC

# För export av modellen/scalern för vidare användning i Streamlit-appen
import joblib
from joblib import Parallel, delayed

# Slutrapport
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Tillfällig (om jag inte glömmer det) sänkning av prioriteten så datorn orkar annat
os.environ['OMP_NUM_THREADS'] = '8' 
os.environ['MKL_NUM_THREADS'] = '8'

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


>>> Tid för denna cell: 3.4 sekunder
>>> Total tid sedan start: 0 minuter och 3 sekunder


## Läs in MNISt-datasetet och splitta det. 

*//Best practice: splitta ut testsetet direkt//*

In [2]:
t0 = time.time()

# Läs in alla MNIST-data
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False, parser='auto')
X = mnist["data"]              
y = mnist["target"].astype(np.uint8)

# Splitta (80/20) med stratifiering för jämna klassfördelningar
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

# Normalisering (en riktigt bra grej för både PCA och SVC!)
# Genom att dela med 255.0 blir alla värden mellan 0 och 1
X_train = X_train / 255.0
X_test = X_test / 255.0

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print("-" * 40)
print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

----------------------------------------
>>> Tid för denna cell: 2.5 sekunder
>>> Total tid sedan start: 0 minuter och 5 sekunder


HoG ska boosta accuracy!

In [3]:
class HogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        def local_hog(img_flat):
            return hog(img_flat.reshape(28, 28), 
                       orientations=self.orientations, 
                       pixels_per_cell=self.pixels_per_cell, 
                       cells_per_block=self.cells_per_block,
                       visualize=False)
        
        return np.array([local_hog(x) for x in X])

Definitioner

In [4]:
t0 = time.time()

def deskew(image):
    # Räta upp lutande siffror 
    img = image.reshape(28, 28)
    
    # Skapa koordinat-matriser (y för rader, x för kolumner)
    y, x = np.mgrid[:28, :28]
    
    # Hitta tyngdpunkten (Center of Mass) med ndimage
    mu = ndimage.center_of_mass(img)
    if np.isnan(mu).any(): # Om bilden är tom
        return img.flatten()
    
    # Beräkna centrala moments (mu11 = kovarians, mu02 = varians i y-led)
    mu11 = np.sum((x - mu[1]) * (y - mu[0]) * img)
    mu02 = np.sum((y - mu[0])**2 * img)
    
    # Om variansen är för liten lutar siffran inte eller är för tunn
    if abs(mu02) < 1e-2:
        return img.flatten()
    
    # Skew-faktorn (förskjutning av x per enhet y)
    skew = mu11 / mu02
    
    # Här rätas x upp genom subtrahering av skew * y
    # Matrisen blir [[1, 0], [skew, 1]] med SciPys omvända ordning.
    matrix = np.array([[1, 0], [skew, 1]])
    
    # Offset för att rotera/skeva kring bildens centrum (14, 14)
    center = np.array([14, 14])
    offset = center - np.dot(matrix, center)
    
    # Transformation
    img_deskewed = ndimage.affine_transform(img, matrix, offset=offset, order=1, mode='constant', cval=0)
    return img_deskewed.flatten()

def shift_image(image, dx, dy):
    return ndimage.shift(image.reshape(28, 28), [dy, dx], cval=0, mode="constant").flatten()

def rotate_image(image, angle):
    return ndimage.rotate(image.reshape(28, 28), angle, reshape=False, cval=0, mode="constant").flatten()

def zoom_image(image, factor):
    rescaled = ndimage.zoom(image.reshape(28, 28), factor)
    if factor > 1.0: # Zooma in (klipp)
        start = int((rescaled.shape[0] - 28) / 2)
        final = rescaled[start:start+28, start:start+28]
    else: # Zooma ut (padda)
        pad = int((28 - rescaled.shape[0]) / 2)
        final = np.pad(rescaled, ((pad, 28-rescaled.shape[0]-pad), (pad, 28-rescaled.shape[1]-pad)), mode='constant')
    return final.flatten()

from scipy.ndimage import gaussian_filter, map_coordinates

def elastic_transform(image, alpha=8, sigma=3.5):
    """
    Kirurgisk Elastic Deformation för MNIST.
    alpha: Kontrollerar intensiteten i deformationen (hur långt pixlarna flyttas).
    sigma: Kontrollerar mjukheten (högre värde = mjukare kurvor, lägre = taggigare).
    """
    # Vi utgår från att bilden kommer in som en platt array (784,)
    shape = (28, 28)
    image_reshaped = image.reshape(shape)
    
    # Skapa ett slumpmässigt fält för förskjutning
    # Vi använder en lokal RandomState för att inte störa globala frön om det behövs
    random_state = np.random.RandomState(None)
    
    # Generera brus och filtrera det med Gaussian för att få mjuka "vågor"
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    # Skapa ett koordinatnät
    x, y = np.mgrid[0:shape[0], 0:shape[1]]
    
    # Mappa om bildens pixlar till de nya koordinaterna
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))
    distorted_image = map_coordinates(image_reshaped, indices, order=1, mode='reflect')
    
    return distorted_image.flatten()

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

>>> Tid för denna cell: 0.0 sekunder
>>> Total tid sedan start: 0 minuter och 5 sekunder


Här kommer lite extra data

In [5]:
t0 = time.time()

# Sökvägar
paths = {
    'custom': 'collected_data'
}

X_extra = []
y_extra = []

def process_and_add(folder, label_pos, split_char='_'):
    added_count = 0
    if not os.path.exists(folder):
        print(f"VARNING: Mappen '{folder}' hittades inte!")
        return 0
    
    files = [f for f in os.listdir(folder) if f.endswith(".png")]
    print(f"Hittade {len(files)} st .png-filer i {folder}...")

    for filename in files:
        try:
            # Extrahera label baserat på filnamnsstruktur
            parts = filename.split(split_char)
            # Hitta siffran i filnamnet
            label = None
            for p in parts:
                if p.isdigit() and len(p) == 1:
                    label = int(p)
                    break
            
            if label is None:
                # Fallback
                label = int(parts[label_pos])
            
            # Bearbeta bilden till MNIST-format
            img = Image.open(os.path.join(folder, filename)).convert('L')
            img = img.resize((28, 28), Image.Resampling.LANCZOS)
            img_array = np.array(img).astype(np.float32) / 255.0
            
            # MNIST-standard: Vit siffra på svart bakgrund
            # Kolla medianen av kanterna
            edge_median = np.median(np.concatenate([img_array[0,:], img_array[-1,:], img_array[:,0], img_array[:,-1]]))
            # Ljus bakgrund -> Invertera
            if edge_median > 0.5: 
                img_array = 1.0 - img_array
                
            X_extra.append(img_array.flatten())
            y_extra.append(label)
            added_count += 1
        except Exception as e:
            print(f"Skippar {filename}: {e}")
            
    return added_count

# Läs in användarskapade bilder
count_c = process_and_add(paths['custom'], label_pos=1)

if len(X_extra) > 0:
    X_train = np.vstack([X_train, np.array(X_extra)])
    y_train = np.concatenate([y_train, np.array(y_extra)])
    print("-" * 30)
    print(f"Nytt antal i X_train: {len(X_train)} (ökning med {len(X_extra)})")
else:
    print("-" * 30)
    print("INGA EXTRA BILDER. Kontrollera mappen!")

# Uppdatera X_train_deskewed med alla bilder
print("Rätar upp det nya kombinerade träningssetet (Deskewing)...")
X_train_deskewed = np.array([deskew(img) for img in X_train])

cell_time = time.time() - t0
print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")

Hittade 18 st .png-filer i collected_data...
------------------------------
Nytt antal i X_train: 56018 (ökning med 18)
Rätar upp det nya kombinerade träningssetet (Deskewing)...
>>> Tid för denna cell: 4.5 sekunder


Hard negative mining

In [6]:
# --- NY CELL 21: HITTA ALLA ÄRLIGA FEL (CROSS-VALIDATION) ---
from sklearn.model_selection import cross_val_predict
import time

t0 = time.time()
print("Letar efter ALLA svåra bilder i träningssetet via Cross-Validation...")

# Vi använder en något snabbare inställning för att inte vänta hela dagen
# Men tillräckligt bra för att hitta de riktiga felen
error_finder_pipe = Pipeline([
    ('pca', PCA(n_components=80, whiten=True)),
    ('scaler', StandardScaler()),
    ('svc', SVC(C=5, kernel='rbf'))
])

# cross_val_predict gör att modellen gissar på bilder den INTE tränat på just då
# cv=3 betyder att den kör 3 rundor. Tar ca 1-2 minuter.
y_train_cv_pred = cross_val_predict(error_finder_pipe, X_train_deskewed, y_train, cv=3, n_jobs=-1)

# Identifiera alla ärliga fel
hard_indices = np.where(y_train_cv_pred != y_train)[0]
X_hard = X_train_deskewed[hard_indices]
y_hard = y_train[hard_indices]

print("-" * 30)
print(f"IDENTIFIERING KLAR: Hittade {len(X_hard)} svåra bilder (ca {len(X_hard)/len(y_train)*100:.2f}% av datat).")
print(f"Detta är en 'ärlig' felmarginal på träningsdatat.")

# Exportera dessa till mappen
if not os.path.exists('mnist_errors'):
    os.makedirs('mnist_errors')

for i, (img_flat, label) in enumerate(zip(X_hard, y_hard)):
    img_array = (img_flat.reshape(28, 28) * 255).astype(np.uint8)
    img = Image.fromarray(img_array)
    filename = f"hard_negative_{label}_idx{i}.png"
    img.save(os.path.join('mnist_errors', filename))

print(f">>> {len(X_hard)} bilder sparade i 'mnist_errors'.")
print(f">>> Tid: {time.time() - t0:.1f} sekunder")

Letar efter ALLA svåra bilder i träningssetet via Cross-Validation...
------------------------------
IDENTIFIERING KLAR: Hittade 852 svåra bilder (ca 1.52% av datat).
Detta är en 'ärlig' felmarginal på träningsdatat.
>>> 852 bilder sparade i 'mnist_errors'.
>>> Tid: 35.1 sekunder


# Augmentering

In [ ]:
"""t0 = time.time()

# Vi börjar med de vanliga bilderna PLUS de svåra bilderna (en extra kopia)
X_base_for_aug = np.vstack([X_train_deskewed, X_hard])
y_base_for_aug = np.concatenate([y_train, y_hard])

X_train_augmented = [image for image in X_base_for_aug]
y_train_augmented = [label for label in y_base_for_aug]

print(f"Startar ultra-augmentering på {len(X_base_for_aug)} bilder (inkl. {len(X_hard)} svåra)...")

# Vi kör dina standard-augmentationer på hela den utökade potten
for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_base_for_aug, y_base_for_aug):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

for angle in (-8, -4, 4, 8):
    for image, label in zip(X_base_for_aug, y_base_for_aug):
        X_train_augmented.append(rotate_image(image, angle))
        y_train_augmented.append(label)

for factor in (0.9, 1.1):
    for image, label in zip(X_base_for_aug, y_base_for_aug):
        X_train_augmented.append(zoom_image(image, factor))
        y_train_augmented.append(label)

# --- OPTIMERAT BLOCK: MÅLINRIKTAD ELASTISK AUGMENTERING ---
t_aug = time.time()
target_classes = [3, 4, 5, 7, 9]
print(f">>> Skapar elastiska kopior för klasserna: {target_classes}...")

# Vi lägger till direkt i de befintliga listorna istället för att vstacka sen
for image, label in zip(X_base_for_aug, y_base_for_aug):
    if label in target_classes:
        # Skapa första elastiska kopian
        X_train_augmented.append(elastic_transform(image, alpha=8, sigma=3.5))
        y_train_augmented.append(label)
        # Skapa andra elastiska kopian
        X_train_augmented.append(elastic_transform(image, alpha=8, sigma=3.5))
        y_train_augmented.append(label)
# NU konverterar vi allt till numpy-arrayer en enda gång (sparar massor av RAM)
print("Konverterar till slutgiltiga arrayer...")
X_train_augmented = np.array(X_train_augmented, dtype='float32')
y_train_augmented = np.array(y_train_augmented)

# Spara till C:
save_path = "C:/mnist_data/mnist_augmented_boosted.joblib"
joblib.dump((X_train_augmented, y_train_augmented), save_path)

print("-" * 40)
print(f"Ny total datamängd: {len(X_train_augmented)} rader") # Kommer vara ca 680 000+
print(f">>> Tid för hela augmenteringen: {time.time() - t0:.1f} sekunder")
"""

>>> Skapar elastiska kopior för klasserna: [3, 4, 5, 7, 9]...


NameError: name 'X_base_for_aug' is not defined

Preparera testsetet så att jämförelsen blir rättvis.

In [8]:
t0 = time.time()

# Här rätas testbilderna upp för att matcha modellen
X_test_deskewed = np.array([deskew(img) for img in X_test])

X_test_deskewed = X_test_deskewed.astype('float32')

cell_time = time.time() - t0
print("-" * 40)
print(f">>> Sökningen klar på {cell_time:.1f} sekunder.")

----------------------------------------
>>> Sökningen klar på 1.2 sekunder.


## Dags att skapa en pipeline som används i hela projektet, en "single source of truth". 

In [9]:
t0 = time.time()

# Detta är vår "Ultra-Pipeline"
# Vi kombinerar Notebook 3:s PCA med form-analys från HOG
final_pipe_ultra = Pipeline([
    ('feature', FeatureUnion([
        ('pca', PCA(n_components=65, whiten=True)), # Från Notebook 3
        ('hog', HogTransformer())      # Vår nya form-förstärkare
    ])),
    ('scaler', StandardScaler()),      # Livsviktig för att balansera PCA och HOG
    ('svc', SVC(
        C=25, 
        kernel='rbf', 
        gamma='scale', 
        cache_size=500,              
        probability=True,
        tol=1e-2,
        shrinking=True, 
        verbose=True
    ))
])

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Pipeline definierad! Tid: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

>>> Pipeline definierad! Tid: 0.0 sekunder
>>> Total tid sedan start: 1 minuter och 41 sekunder


In [10]:
"""# --- CELL 28: OPTIMERAD TRÄNING (MINNESSNÅL VERSION) ---
import gc 
import time

t0 = time.time()

# 1. Förbered data UTAN att skapa en kopia
# Vi hoppar över .astype('float32') eftersom Cell 25 redan gjorde det jobbet.
# På så sätt sparar vi ca 2.5 GB RAM omedelbart.
X_train_final = X_train_augmented 
y_train_final = y_train_augmented

# 2. RADIKAL MINNESRENSNING
# Vi tar bort referensen till listan och tvingar Python att frigöra RAM
if 'X_train_augmented' in locals():
    del X_train_augmented
gc.collect() 

# 3. FINJUSTERA PIPELINEN FÖR ATT KLARA ZOOM + 681k RADER
# Vi sänker cache_size drastiskt för att ge plats åt probability-kalkylen.
final_pipe_ultra.set_params(
    feature__pca__n_components=140,
    feature__pca__whiten=True,
    svc__C=25, 
    svc__cache_size=1000,         # Sänkt från 3000 för att rädda RAM
    svc__probability=True,       # Måste vara med här för Soft Voting!
    svc__tol=1e-3,               
    svc__shrinking=True,         # Hjälper till att hålla nere antalet support-vektorer
    svc__verbose=True
)

print(f"\n>>> STARTAR TRÄNING: {len(X_train_final)} rader.")

try:
    # Denna rad gör det tunga jobbet (räkna med 45-60 minuter pga färre trådar)
    final_pipe_ultra.fit(X_train_final, y_train_final)
    print("\nSUCCESS! Modellen är tränad.")
except MemoryError:
    print("\nAI-Haveri: RAM-minnet räckte inte till. Stäng Trados/Zoom och prova igen.")
    gc.collect()

# 4. SLUTGILTIG UTVÄRDERING
print("\nGenomför slutexamen på testdata...")
base_accuracy = final_pipe_ultra.score(X_test_deskewed, y_test)
y_pred_tmp = final_pipe_ultra.predict(X_test_deskewed)
num_errors = np.sum(y_pred_tmp != y_test)

print("\n" + "="*40)
print(f"RESULTAT: BOOSTED SVC")
print("-" * 40)
print(f"Accuracy:      {base_accuracy:.5f}")
print(f"Antal fel:     {num_errors} av {len(y_test)}")
print("="*40)

cell_time = time.time() - t0
print(f">>> Full träning och test klar på: {cell_time/60:.1f} minuter")
"""

'# --- CELL 28: OPTIMERAD TRÄNING (MINNESSNÅL VERSION) ---\nimport gc \nimport time\n\nt0 = time.time()\n\n# 1. Förbered data UTAN att skapa en kopia\n# Vi hoppar över .astype(\'float32\') eftersom Cell 25 redan gjorde det jobbet.\n# På så sätt sparar vi ca 2.5 GB RAM omedelbart.\nX_train_final = X_train_augmented \ny_train_final = y_train_augmented\n\n# 2. RADIKAL MINNESRENSNING\n# Vi tar bort referensen till listan och tvingar Python att frigöra RAM\nif \'X_train_augmented\' in locals():\n    del X_train_augmented\ngc.collect() \n\n# 3. FINJUSTERA PIPELINEN FÖR ATT KLARA ZOOM + 681k RADER\n# Vi sänker cache_size drastiskt för att ge plats åt probability-kalkylen.\nfinal_pipe_ultra.set_params(\n    feature__pca__n_components=140,\n    feature__pca__whiten=True,\n    svc__C=25, \n    svc__cache_size=1000,         # Sänkt från 3000 för att rädda RAM\n    svc__probability=True,       # Måste vara med här för Soft Voting!\n    svc__tol=1e-3,               \n    svc__shrinking=True,       

In [11]:
# --- NY CELL: BASELINE-TRÄNING (UTAN AUGMENTERING) ---
import gc 
import time

t0 = time.time()

# 1. Använd endast originaldata (56 000 rader) som rätats upp (Deskewed)
# Vi hoppar över de 681 000 augmenterade raderna helt.
X_train_final = X_train_deskewed.astype('float32')
y_train_final = y_train

# 2. RENSA MINNET
# Vi tar bort eventuella rester av det stora augmenterade setet
if 'X_train_augmented' in locals():
    del X_train_augmented
gc.collect() 

# 3. KONFIGURERA PIPELINEN FÖR BASELINE
# Vi använder n=65 eftersom din sweep visade att det är mest effektivt.
final_pipe_ultra.set_params(
    feature__pca__n_components=65, 
    feature__pca__whiten=True,
    svc__C=25, 
    svc__probability=True,         # Vi behåller True så du kan använda den i juryn sen
    svc__cache_size=2000,          # Kan vara högre nu när vi har färre rader
    svc__verbose=True
)

print(f">>> STARTAR BASELINE-TRÄNING: {len(X_train_final)} rader.")
print("Detta bör ta ca 1-2 minuter istället för 10+.")

# 4. TRÄNA
final_pipe_ultra.fit(X_train_final, y_train_final)

print(f"\nSUCCESS! Baseline-modellen klar på {(time.time() - t0)/60:.1f} minuter.")

>>> STARTAR BASELINE-TRÄNING: 56018 rader.
Detta bör ta ca 1-2 minuter istället för 10+.

SUCCESS! Baseline-modellen klar på 0.2 minuter.


In [13]:
# --- NY CELL: BASELINE-RESULTAT ---

# 1. Beräkna accuracy direkt
baseline_acc = final_pipe_ultra.score(X_test_deskewed, y_test)

# 2. Beräkna antal fel
y_pred = final_pipe_ultra.predict(X_test_deskewed)
num_errors = np.sum(y_pred != y_test)

# 3. Presentera resultatet
print("="*30)
print(f"BASELINE SVC RESULTAT")
print("-"*30)
print(f"Accuracy:  {baseline_acc:.5f}")
print(f"Antal fel: {num_errors} av {len(y_test)}")
print("="*30)

BASELINE SVC RESULTAT
------------------------------
Accuracy:  0.98964
Antal fel: 145 av 14000


Dumpa ned modellen för voting

In [12]:
"""# --- CELL 12: EXPORTERA MASTER-SVC TILL C-DISKEN ---
import os

# 1. Definiera den exakta sökvägen till din snabba disk
# Vi använder forward slashes (/) eller dubbla backslashes (\\) för Windows-sökvägar i Python
model_path = 'C:/mnist_data/mnist_svc_hog_max.joblib'

# 2. Säkerställ att mappen finns (ifall den raderats av misstag)
if not os.path.exists('C:/mnist_data'):
    os.makedirs('C:/mnist_data')
    print(">>> Skapade mappen C:/mnist_data")

# 3. Spara modellen (skriver över den gamla filen automatiskt)
print(f"Sparar din boostade SVC-expert till {model_path}...")
joblib.dump(final_pipe_ultra, model_path)

print("-" * 40)
print("KLART! Din starkaste SVC-modell hittills är nu säkrad på C-disken.")
print("Den är redo att agera 'ordförande' i din ensemble-jury.")
"""

'# --- CELL 12: EXPORTERA MASTER-SVC TILL C-DISKEN ---\nimport os\n\n# 1. Definiera den exakta sökvägen till din snabba disk\n# Vi använder forward slashes (/) eller dubbla backslashes (\\) för Windows-sökvägar i Python\nmodel_path = \'C:/mnist_data/mnist_svc_hog_max.joblib\'\n\n# 2. Säkerställ att mappen finns (ifall den raderats av misstag)\nif not os.path.exists(\'C:/mnist_data\'):\n    os.makedirs(\'C:/mnist_data\')\n    print(">>> Skapade mappen C:/mnist_data")\n\n# 3. Spara modellen (skriver över den gamla filen automatiskt)\nprint(f"Sparar din boostade SVC-expert till {model_path}...")\njoblib.dump(final_pipe_ultra, model_path)\n\nprint("-" * 40)\nprint("KLART! Din starkaste SVC-modell hittills är nu säkrad på C-disken.")\nprint("Den är redo att agera \'ordförande\' i din ensemble-jury.")\n'